In [ ]:

import seaborn as sns
%matplotlib inline

In [ ]:
from jupyterworkflow.data import get_fremont_data
import pandas as pd

In [ ]:
def test_fremont_data():
    data = get_fremont_data()
    assert all(data.columns == ['East', 'West', 'Total'])
    assert isinstance(data.index,pd.DatetimeIndex)

In [ ]:
%%time
data = pd.read_csv('Fremont.csv', index_col='Date')
data.columns = ['East', 'West']
data['Total'] = data['East'] + data['West']
try: 
    data.index = pd.to_datetime(data.index, format = '%m/%d/%Y %H:%M:%S %p')
except TypeError:
    data.index = pd.to_datetime(data.index)


In [ ]:
import numpy as np

In [ ]:
!make test

In [ ]:
!head Fremont.csv

In [ ]:

data.plot();

In [ ]:
data.resample('W').sum().plot();

In [ ]:
data.resample('Y').sum().plot();

In [ ]:
ax = data.resample('D').sum().rolling(365).sum().plot()
ax.set_ylim(0,None);

In [ ]:
data.groupby(data.index.time).mean().plot();

In [ ]:
pivoted_table = data.pivot_table('Total', index = data.index.time, columns = data.index.date)
pivoted_table.head()

In [ ]:
pivoted_table.plot(legend = False, alpha = 0.01);

In [ ]:
print('The notebook was executed')

In [ ]:
with open('foo.txt', 'w') as file:
    file.write('It worked')